## Using KerasNeuralFinder

In [10]:
#Importing the necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
import time
import datetime
from sklearn.preprocessing import MinMaxScaler
import IPython
import os
import math
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc

In [11]:
import KerasOptimizerNN
from KerasOptimizerNN import KerasNeuralFinder

In [12]:
X_temp = pd.read_csv("X_temp.csv").values
y_temp = pd.read_csv("y_temp.csv").values

In [13]:
X_train = pd.read_csv("X_train.csv").values
y_train = pd.read_csv("y_train.csv").values

In [14]:
y_temp = y_temp[:,-1]
y_train = y_train[:,-1]

In [15]:
param_grid = {}
param_grid['hidden_layer_neurons'] =[ [11, 8, 6], [8,5]]
param_grid['output_layer_neurons'] = [1]
param_grid['hidden_layer_activations'] = ['sigmoid', 'relu']
param_grid['output_layer_activations'] = ['sigmoid']
param_grid['kernel_inializers'] = ['glorot_normal']
param_grid['bias_initializers'] = ['glorot_normal']
param_grid['optimizers'] = [ 'Adam']
param_grid['epochs'] = [20]
param_grid['learning_rate'] = [0.01]
param_grid['loss_functions'] = ['binary_crossentropy']

### New stuff!!!

* Can provide different additional metrics (built-in only at the moment) like binary_accuracy
* Mean and the min/max based on the confidence interval provided by user are returned in results
* get_best_results() method implemented get the best out of the run. Example usage below.
* Can choose to retain weights for future re-run using flag - retain_weights
* Can choose to reuse already stored weights to get same results

#### Roadmap

* To be able to store weights for more than 1 historical run
* Then we shall be able to choose the best weights as well, along with the best neural network design
* To implement parallel processing using multiprocessing library


In [16]:
knf = KerasNeuralFinder()
results = knf.fit(param_grid=param_grid, store_results=True, silent_mode=True,
                  X=X_train, y=y_train, fold=5,
                  confidence_interval=80,
                  metrics=['binary_accuracy', 'accuracy'])

Choice 4/4..., epoch = 20, time_taken=9.88 seconds, loss=69.5123338592372, accuracy=49.629610776901245
Parameters are :

{'hidden_layer_neurons': [8, 5]}
{'output_layer_neurons': 1}
{'hidden_layer_activations': 'relu'}
{'output_layer_activations': 'sigmoid'}
{'kernel_inializers': 'glorot_normal'}
{'bias_initializers': 'glorot_normal'}
{'optimizers': 'Adam'}
{'epochs': 20}
{'learning_rate': 0.01}
{'loss_functions': 'binary_crossentropy'}

Completed running ... overall time taken = 41.53


In [17]:
results.T

,0,1,2,3
hidden_layer_neurons,"11,8,6","11,8,6","8,5","8,5"
output_layer_neurons,1,1,1,1
hidden_layer_activations,sigmoid,relu,sigmoid,relu
output_layer_activations,sigmoid,sigmoid,sigmoid,sigmoid
kernel_inializers,glorot_normal,glorot_normal,glorot_normal,glorot_normal
bias_initializers,glorot_normal,glorot_normal,glorot_normal,glorot_normal
optimizers,Adam,Adam,Adam,Adam
epochs,20,20,20,20
learning_rate,0.01,0.01,0.01,0.01
loss_functions,binary_crossentropy,binary_crossentropy,binary_crossentropy,binary_crossentropy


In [19]:
knf.get_best_result(results, by={'accuracy': 'high'})

,hidden_layer_neurons,output_layer_neurons,hidden_layer_activations,output_layer_activations,kernel_inializers,bias_initializers,optimizers,epochs,learning_rate,loss_functions,...,max_accuracy(80%),val_loss,min_val_loss(80%),max_val_loss(80%),val_binary_accuracy,min_val_binary_accuracy(80%),max_val_binary_accuracy(80%),val_accuracy,min_val_accuracy(80%),max_val_accuracy(80%)
1,"11,8,6",1,relu,sigmoid,glorot_normal,glorot_normal,Adam,20,0.01,binary_crossentropy,...,74.47,65.875345,58.67,73.08,57.988155,44.71,71.26,57.988155,44.71,71.26
